## Work on `fct_monthly_operator_summary`
* Operator grain.

In [1]:
from datetime import datetime
from functools import cache
from pathlib import Path
from typing import Literal

import _sql_query
import geopandas as gpd
import pandas as pd
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from calitp_data_analysis.gcs_pandas import GCSPandas

In [2]:
from update_vars import (
    GTFS_DATA_DICT,
    analysis_month,
    file_name,
    last_year,
    previous_month,
)

In [3]:
import _portfolio_charts

In [4]:
from omegaconf import OmegaConf

readable_dict = OmegaConf.load("new_readable.yml")

In [5]:
@cache
def gcs_pandas():
    return GCSPandas()

In [6]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [7]:
analysis_name = "Alameda-Contra Costa Transit District"

In [8]:
GCS_PATH = f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/"

In [9]:
operator_summary_url = f"{GCS_PATH}{GTFS_DATA_DICT.gtfs_digest_rollup.operator_summary}_{file_name}.parquet"

In [10]:
GTFS_DATA_DICT.gtfs_digest_rollup.operator_summary

'fct_monthly_operator_summary'

In [11]:
operator_df = gcs_pandas().read_parquet(
    operator_summary_url,
    filters=[("Day Type", "==", "Weekday"), ("Analysis Name", "==", analysis_name)],
)

In [12]:
operator_all_df = gcs_pandas().read_parquet(operator_summary_url)

In [14]:
operator_df.sort_values(by = ["Date"])

,Date,Analysis Name,Caltrans District,VP Name,TU Name,N Trips,Day Type,Daily Trips,Ttl Service Hours,N Routes,N Days,N Shapes,N Stops,VP Messages Per Minute,N VP Trips,Daily VP Trips,Pct VP Trips,N VP Routes,Pct VP Service Hours,TU Messages Per Minute,N TU Trips,Daily TU Trips,Pct TU Trips,N TU Routes,Pct TU Service Hours,Percent of Trips with Trip Updates,Percent of Trips with Vehicle Positions
2,2024-12-01,Alameda-Contra Costa Transit District,04-Bay Area / Oakland,Bay Area 511 AC Transit Vehicle Position,Bay Area 511 AC Transit Trip Updates,112864,Weekday,5374.50,84309.20,113.57,21,279.76,4534.29,3.00,105595,5028.30,0.94,1.00,1.24,3.00,110736,5273.10,0.98,1.00,2.48,98.11,93.56
21,2024-12-01,Alameda-Contra Costa Transit District,04-Bay Area / Oakland,Bay Area 511 Dumbarton Express VehiclePositions,Bay Area 511 Dumbarton Express TripUpdates,1554,Weekday,74.00,1341.90,2.00,21,6.00,72.00,2.80,1219,58.00,0.79,1.00,0.86,2.50,1301,62.00,0.84,1.00,2.24,83.72,78.44
6,2025-01-01,Alameda-Contra Costa Transit District,04-Bay Area / Oakland,Bay Area 511 AC Transit Vehicle Position,Bay Area 511 AC Transit Trip Updates,122954,Weekday,5345.80,92083.20,118.35,23,293.61,4583.39,3.00,113515,4935.40,0.93,1.00,1.18,2.90,119742,5206.20,0.97,1.00,2.31,97.39,92.32
15,2025-01-01,Alameda-Contra Costa Transit District,04-Bay Area / Oakland,Bay Area 511 Dumbarton Express VehiclePositions,Bay Area 511 Dumbarton Express TripUpdates,1628,Weekday,74.00,1405.80,2.00,22,6.00,72.00,2.80,1261,57.30,0.77,1.00,0.81,2.50,1388,63.10,0.85,1.00,2.22,85.26,77.46
1,2025-02-01,Alameda-Contra Costa Transit District,04-Bay Area / Oakland,Bay Area 511 AC Transit Vehicle Position,Bay Area 511 AC Transit Trip Updates,102639,Weekday,5402.10,76905.00,123.26,19,305.47,4643.84,3.00,95643,5033.80,0.93,1.00,1.22,3.00,100164,5271.80,0.98,1.00,2.39,97.59,93.18
23,2025-02-01,Alameda-Contra Costa Transit District,04-Bay Area / Oakland,Bay Area 511 Dumbarton Express VehiclePositions,Bay Area 511 Dumbarton Express TripUpdates,1406,Weekday,74.00,1214.10,2.00,19,6.00,72.00,2.80,1124,59.20,0.80,1.00,0.89,2.60,1226,64.50,0.87,1.00,2.32,87.20,79.94
20,2025-03-01,Alameda-Contra Costa Transit District,04-Bay Area / Oakland,Bay Area 511 Dumbarton Express VehiclePositions,Bay Area 511 Dumbarton Express TripUpdates,1406,Weekday,74.00,1214.10,2.00,19,6.00,72.00,2.80,1054,55.50,0.75,1.00,0.83,2.60,1173,61.70,0.83,1.00,2.21,83.43,74.96
10,2025-03-01,Alameda-Contra Costa Transit District,04-Bay Area / Oakland,Bay Area 511 AC Transit Vehicle Position,Bay Area 511 AC Transit Trip Updates,115010,Weekday,5476.70,86312.70,127.33,21,316.52,4700.38,3.00,109795,5228.30,0.95,1.00,1.25,3.00,113367,5398.40,0.99,1.00,2.37,98.57,95.47
11,2025-04-01,Alameda-Contra Costa Transit District,04-Bay Area / Oakland,Bay Area 511 AC Transit Vehicle Position,Bay Area 511 AC Transit Trip Updates,119871,Weekday,5448.70,90095.80,119.41,22,296.36,4626.50,3.00,114329,5196.80,0.95,1.00,1.25,3.00,117734,5351.50,0.98,1.00,2.38,98.22,95.38
24,2025-04-01,Alameda-Contra Costa Transit District,04-Bay Area / Oakland,Bay Area 511 Dumbarton Express VehiclePositions,Bay Area 511 Dumbarton Express TripUpdates,1628,Weekday,74.00,1405.80,2.00,22,6.00,72.00,2.80,1230,55.90,0.76,1.00,0.83,2.60,1326,60.30,0.81,1.00,2.16,81.45,75.55


### Operator Grain: TU Messages per Minute
* Update legend to have a more dramatic color scheme?

In [ ]:
def create_tu_minute(df: pd.DataFrame):

    chart_dict = readable_dict.trip_updates_minute

    chart = _portfolio_charts.circle_chart(
        df=df,
        x_col="Date",
        y_col="TU Messages Per Minute",
        color_col="TU Messages Per Minute",
        color_scheme=[*chart_dict.colors],
        tooltip_cols=list(chart_dict.tooltip),
        date_format="%b %Y",
        y_ticks=chart_dict.ticks,
    )

    chart = _portfolio_charts.configure_chart(
        chart,
        width=400,
        height=250,
        title=f"{chart_dict.title}",
        subtitle=chart_dict.subtitle,
    )

    return chart

In [ ]:
create_tu_minute(operator_df)

In [ ]:
tu_df_min = _portfolio_charts.circle_chart(
    df=operator_df,
    x_col="Date",
    y_col="TU Messages Per Minute",
    color_col="TU Messages Per Minute",
    color_scheme=["#ffb81c", "#00b2a9"],
    tooltip_cols=["Date", "TU Messages Per Minute"],
    date_format="%b %Y",
)

In [ ]:
_portfolio_charts.configure_chart(
    tu_df_min,
    width=400,
    height=250,
    title="Trip Update Messages per Minute",
    subtitle="Based on Weekday Data, across all routes.",
)

### Operator Grain - Vehicle Positions per Minute

In [ ]:
vp_df = _portfolio_charts.circle_chart(
    df=operator_df,
    x_col="Date",
    y_col="VP Messages Per Minute",
    color_col="VP Messages Per Minute",
    color_scheme=["#b1e4e3", "#4b4f54"],
    tooltip_cols=["Date", "VP Messages Per Minute"],
    date_format="%b %Y",
)

In [ ]:
_portfolio_charts.configure_chart(
    vp_df,
    width=400,
    height=250,
    title="Vehicle Positions per Minute",
    subtitle="Based on Weekday Data",
)

In [ ]:
def create_vp_minute(df: pd.DataFrame):

    chart_dict = readable_dict.vehicle_positions_minute

    chart = _portfolio_charts.circle_chart(
        df=df,
        x_col="Date",
        y_col="VP Messages Per Minute",
        color_col="VP Messages Per Minute",
        color_scheme=[*chart_dict.colors],
        tooltip_cols=list(chart_dict.tooltip),
        date_format="%b %Y",
        y_ticks=chart_dict.ticks,
    )

    chart = _portfolio_charts.configure_chart(
        chart,
        width=400,
        height=250,
        title=f"{chart_dict.title}",
        subtitle=chart_dict.subtitle,
    )

    return chart

In [ ]:
create_vp_minute(operator_df)

### Operator Grain: % of Trips with Trip Updates

#### Can't get colors for line charts to work in my original code.

In [ ]:
_portfolio_charts.bar_chart(
    df=operator_df,
    x_col="Date",
    y_col="Percent of Trips with Trip Updates",
    color_col="Percent of Trips with Trip Updates",
    color_scheme=["#00b2a9", "#ffb81c", "#b1e4e3", "#d9d9d6"],
    tooltip_cols=["Date", "Percent of Trips with Trip Updates"],
    date_format="%b %Y",
    y_ticks=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
)

In [ ]:
def create_tu_pct(df: pd.DataFrame):

    chart_dict = readable_dict.trip_update_pct

    chart = _portfolio_charts.bar_chart(
        df=df,
        x_col="Date",
        y_col="Percent of Trips with Trip Updates",
        color_col="Percent of Trips with Trip Updates",
        color_scheme=[*chart_dict.colors],
        tooltip_cols=list(chart_dict.tooltip),
        date_format="%b %Y",
        y_ticks=chart_dict.ticks,
    )

    chart = _portfolio_charts.configure_chart(
        chart,
        width=400,
        height=250,
        title=f"{chart_dict.title}",
        subtitle=chart_dict.subtitle,
    )

    return chart

In [ ]:
create_tu_pct(operator_df)

In [ ]:
_portfolio_charts.bar_chart(
    df=operator_df,
    x_col="Date",
    y_col="Percent of Trips with Vehicle Positions",
    color_col="Percent of Trips with Vehicle Positions",
    color_scheme=["#00b2a9", "#ffb81c", "#b1e4e3", "#d9d9d6"],
    tooltip_cols=["Date", "Percent of Trips with Vehicle Positions"],
    date_format="%b %Y",
    y_ticks=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
)

In [ ]:
def create_vp_pct(df: pd.DataFrame):

    chart_dict = readable_dict.vehicle_positions_pct

    chart = _portfolio_charts.bar_cart(
        df=df,
        x_col="Date",
        y_col="Percent of Trips with Vehicle Positions",
        color_col="Percent of Trips with Vehicle Positions",
        color_scheme=[*chart_dict.colors],
        tooltip_cols=list(chart_dict.tooltip),
        date_format="%b %Y",
        y_ticks=chart_dict.ticks,
    )

    chart = _portfolio_charts.configure_chart(
        chart,
        width=400,
        height=250,
        title=f"{chart_dict.title}",
        subtitle=chart_dict.subtitle,
    )

    return chart